# ⚛️ React Frontend Concepts - Greek Derby RAG Chatbot

## Learning Objectives
By the end of this lesson, you will understand:
- Modern React patterns and best practices
- TypeScript integration for type safety
- Context API and useReducer for state management
- Custom hooks for reusable logic
- Component composition and architecture
- API integration patterns
- Performance optimization techniques
- Modern CSS and responsive design

---

## Q1: What modern React patterns do we use in our Greek Derby chatbot?

**Answer:**

Our Greek Derby chatbot frontend showcases several modern React patterns that represent current best practices in React development. Let's explore the key patterns we use:

### 1. **React 19 with TypeScript**
```typescript
// Modern React with TypeScript
import { createContext, useContext, useReducer, type ReactNode } from 'react';

// Type-safe component props
interface MessageProps {
  message: MessageType;
  isError?: boolean;
}
```

**Why this matters:**
- **Type Safety**: Catch errors at compile time
- **Better IDE Support**: Autocomplete and refactoring
- **Self-Documenting Code**: Types serve as documentation
- **Easier Maintenance**: Refactoring is safer and faster

### 2. **Component Composition Pattern**
```typescript
// App.tsx - Main orchestrator
function App() {
  return (
    <ChatProvider>
      <div className="container">
        <Header />
        <Chat />
      </div>
    </ChatProvider>
  );
}

// Chat.tsx - Composed of smaller components
export function Chat() {
  return (
    <div className="chat-container">
      <div className="chat-messages">
        {messages.map((message) => (
          <Message key={message.id} message={message} />
        ))}
      </div>
      <ChatInput />
      <SampleQuestions />
    </div>
  );
}
```

**Benefits:**
- **Reusability**: Components can be used in different contexts
- **Testability**: Each component can be tested in isolation
- **Maintainability**: Changes to one component don't affect others
- **Readability**: Clear component hierarchy and responsibilities

### 3. **Custom Hooks for Logic Reuse**
```typescript
// useApi.ts - Reusable API logic
export function useSendMessage() {
  const { data, loading, error, execute } = useApiCall<ChatResponse>();
  
  const sendMessage = useCallback(async (question: string) => {
    return execute(() => apiService.sendMessage(question));
  }, [execute]);
  
  return { response: data, loading, error, sendMessage };
}

// useScrollToBottom.ts - Reusable scroll logic
export function useScrollToBottom() {
  const scrollContainerRef = useRef<HTMLDivElement>(null);
  const [showScrollButton, setShowScrollButton] = useState(false);
  
  const scrollToBottom = useCallback(() => {
    if (scrollContainerRef.current) {
      scrollContainerRef.current.scrollTop = 
        scrollContainerRef.current.scrollHeight;
    }
  }, []);
  
  return { scrollContainerRef, showScrollButton, scrollToBottom };
}
```

**Why Custom Hooks:**
- **Separation of Concerns**: Logic separate from UI
- **Reusability**: Same logic across multiple components
- **Testability**: Hooks can be tested independently
- **Clean Components**: Components focus on rendering, not logic

### 4. **Context + useReducer Pattern**
```typescript
// ChatContext.tsx - Global state management
export function ChatProvider({ children }: ChatProviderProps) {
  const [state, dispatch] = useReducer(chatReducer, initialState);
  
  const addMessage = (text: string, sender: 'user' | 'bot', isError = false) => {
    const message: Message = {
      id: generateId(),
      text,
      sender,
      timestamp: new Date().toLocaleTimeString('el-GR'),
      isError
    };
    dispatch({ type: 'ADD_MESSAGE', payload: message });
  };
  
  return (
    <ChatContext.Provider value={{ state, dispatch, addMessage, ... }}>
      {children}
    </ChatContext.Provider>
  );
}
```

**Advantages over useState:**
- **Complex State**: Better for multiple related state values
- **Predictable Updates**: Reducer ensures consistent state changes
- **Debugging**: Easy to trace state changes
- **Performance**: Only re-renders when necessary

### 5. **Service Layer Pattern**
```typescript
// api.ts - Centralized API calls
export const apiService = {
  async sendMessage(question: string): Promise<ChatResponse> {
    return apiRequest<ChatResponse>('/chat', {
      method: 'POST',
      body: JSON.stringify({ question }),
    });
  },
  
  async getSampleQuestions(): Promise<SampleQuestionsResponse> {
    return apiRequest<SampleQuestionsResponse>('/sample-questions');
  }
};
```

**Benefits:**
- **Single Responsibility**: Each service handles one concern
- **Error Handling**: Centralized error management
- **Type Safety**: Full TypeScript integration
- **Maintainability**: Easy to modify API calls


## Q2: How do we implement state management with Context API and useReducer?

**Answer:**

Our Greek Derby chatbot uses a sophisticated state management approach combining React Context API with useReducer. This pattern provides a clean, scalable solution for managing complex application state without external libraries like Redux.

### Why Context + useReducer Instead of useState?

**useState Limitations:**
```typescript
// With useState - becomes messy with complex state
const [messages, setMessages] = useState<Message[]>([]);
const [isLoading, setIsLoading] = useState(false);
const [error, setError] = useState<string | null>(null);
const [sampleQuestions, setSampleQuestions] = useState<string[]>([]);

// Multiple setState calls for one action
const handleSendMessage = async (question: string) => {
  setIsLoading(true);
  setError(null);
  try {
    const response = await apiService.sendMessage(question);
    setMessages(prev => [...prev, response]);
  } catch (err) {
    setError(err.message);
  } finally {
    setIsLoading(false);
  }
};
```

**useReducer Solution:**
```typescript
// Clean, predictable state updates
const [state, dispatch] = useReducer(chatReducer, initialState);

const handleSendMessage = async (question: string) => {
  dispatch({ type: 'SET_LOADING', payload: true });
  try {
    const response = await apiService.sendMessage(question);
    dispatch({ type: 'ADD_MESSAGE', payload: response });
  } catch (err) {
    dispatch({ type: 'SET_ERROR', payload: err.message });
  } finally {
    dispatch({ type: 'SET_LOADING', payload: false });
  }
};
```

### Our State Structure:

```typescript
// types/index.ts
export interface ChatState {
  messages: Message[];
  isLoading: boolean;
  sampleQuestions: string[];
  error: string | null;
}

export interface Message {
  id: string;
  text: string;
  sender: 'user' | 'bot';
  timestamp: string;
  isError?: boolean;
}
```

### Reducer Implementation:

```typescript
// ChatContext.tsx
type ChatAction =
  | { type: 'ADD_MESSAGE'; payload: Message }
  | { type: 'SET_LOADING'; payload: boolean }
  | { type: 'SET_SAMPLE_QUESTIONS'; payload: string[] }
  | { type: 'SET_ERROR'; payload: string | null }
  | { type: 'CLEAR_MESSAGES' }
  | { type: 'CLEAR_ERROR' };

function chatReducer(state: ChatState, action: ChatAction): ChatState {
  switch (action.type) {
    case 'ADD_MESSAGE':
      return {
        ...state,
        messages: [...state.messages, action.payload],
        error: null, // Clear error when adding new message
      };
    
    case 'SET_LOADING':
      return {
        ...state,
        isLoading: action.payload,
      };
    
    case 'SET_SAMPLE_QUESTIONS':
      return {
        ...state,
        sampleQuestions: action.payload,
      };
    
    case 'SET_ERROR':
      return {
        ...state,
        error: action.payload,
        isLoading: false, // Stop loading on error
      };
    
    case 'CLEAR_MESSAGES':
      return {
        ...state,
        messages: [initialState.messages[0]], // Keep welcome message
        error: null,
      };
    
    case 'CLEAR_ERROR':
      return {
        ...state,
        error: null,
      };
    
    default:
      return state;
  }
}
```

### Context Provider Setup:

```typescript
// ChatContext.tsx
const ChatContext = createContext<ChatContextType | undefined>(undefined);

export function ChatProvider({ children }: ChatProviderProps) {
  const [state, dispatch] = useReducer(chatReducer, initialState);
  
  // Action creators
  const addMessage = useCallback((text: string, sender: 'user' | 'bot', isError = false) => {
    const message: Message = {
      id: generateId(),
      text,
      sender,
      timestamp: new Date().toLocaleTimeString('el-GR', { 
        hour: '2-digit', 
        minute: '2-digit' 
      }),
      isError
    };
    dispatch({ type: 'ADD_MESSAGE', payload: message });
  }, []);
  
  const setLoading = useCallback((loading: boolean) => {
    dispatch({ type: 'SET_LOADING', payload: loading });
  }, []);
  
  const setError = useCallback((error: string | null) => {
    dispatch({ type: 'SET_ERROR', payload: error });
  }, []);
  
  const clearMessages = useCallback(() => {
    dispatch({ type: 'CLEAR_MESSAGES' });
  }, []);
  
  const value = {
    state,
    dispatch,
    addMessage,
    setLoading,
    setError,
    clearMessages,
  };
  
  return (
    <ChatContext.Provider value={value}>
      {children}
    </ChatContext.Provider>
  );
}
```

### Custom Hook for Context:

```typescript
// ChatContext.tsx
export function useChat() {
  const context = useContext(ChatContext);
  if (context === undefined) {
    throw new Error('useChat must be used within a ChatProvider');
  }
  return context;
}
```

### Using the Context in Components:

```typescript
// Chat.tsx
export function Chat() {
  const { state, addMessage, setLoading, setError } = useChat();
  const { response, loading, error, sendMessage } = useSendMessage();
  
  const handleSendMessage = async (question: string) => {
    // Add user message immediately
    addMessage(question, 'user');
    
    try {
      setLoading(true);
      const response = await sendMessage(question);
      addMessage(response.answer, 'bot');
    } catch (err) {
      addMessage('Sorry, something went wrong. Please try again.', 'bot', true);
      setError(err.message);
    } finally {
      setLoading(false);
    }
  };
  
  return (
    <div className="chat-container">
      {/* Render messages */}
    </div>
  );
}
```

### Benefits of This Approach:

1. **Predictable State Updates**: All state changes go through the reducer
2. **Centralized Logic**: State management logic is in one place
3. **Type Safety**: Full TypeScript support for actions and state
4. **Debugging**: Easy to trace state changes with action types
5. **Performance**: Only components using specific state re-render
6. **Testability**: Reducer can be tested independently
7. **Scalability**: Easy to add new state properties and actions

### When to Use This Pattern:

**Good for:**
- Complex state with multiple related values
- State that needs to be shared across many components
- Applications that need predictable state updates
- When you want to avoid external state management libraries

**Consider alternatives for:**
- Very simple state (use useState)
- Server state (use React Query or SWR)
- Very large applications (consider Redux Toolkit)


## Q3: How do we create custom hooks for reusable logic?

**Answer:**

Custom hooks are one of the most powerful patterns in modern React. They allow us to extract component logic into reusable functions, making our code more modular, testable, and maintainable. Our Greek Derby chatbot uses several custom hooks to handle different concerns.

### What are Custom Hooks?

Custom hooks are JavaScript functions that:
- Start with "use" (React convention)
- Can call other hooks
- Allow you to extract component logic into reusable functions
- Return values that components can use

### Our Custom Hooks:

#### 1. **useApi Hook - Generic API Logic**

```typescript
// useApi.ts
function useApiCall<T>() {
  const [data, setData] = useState<T | null>(null);
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState<string | null>(null);

  const execute = useCallback(async (apiCall: () => Promise<T>) => {
    setLoading(true);
    setError(null);
    
    try {
      const result = await apiCall();
      setData(result);
      return result;
    } catch (err) {
      const errorMessage = err instanceof ApiError 
        ? err.message 
        : 'An unexpected error occurred';
      setError(errorMessage);
      throw err;
    } finally {
      setLoading(false);
    }
  }, []);

  const reset = useCallback(() => {
    setData(null);
    setError(null);
    setLoading(false);
  }, []);

  return { data, loading, error, execute, reset };
}
```

**Why this pattern:**
- **Reusable**: Can be used for any API call
- **Type Safe**: Generic `<T>` provides type safety
- **Consistent**: Same loading/error handling everywhere
- **Testable**: Easy to test in isolation

#### 2. **useSendMessage Hook - Specific API Logic**

```typescript
// useApi.ts
export function useSendMessage() {
  const { data, loading, error, execute } = useApiCall<ChatResponse>();

  const sendMessage = useCallback(async (question: string) => {
    return execute(() => apiService.sendMessage(question));
  }, [execute]);

  return { 
    response: data, 
    loading, 
    error, 
    sendMessage 
  };
}

export function useSampleQuestions() {
  const { data, loading, error, execute } = useApiCall<SampleQuestionsResponse>();

  const loadSampleQuestions = useCallback(async () => {
    return execute(() => apiService.getSampleQuestions());
  }, [execute]);

  return { 
    sampleQuestions: data?.sample_questions || [], 
    loading, 
    error, 
    loadSampleQuestions 
  };
}
```

**Benefits:**
- **Separation of Concerns**: API logic separate from UI
- **Composability**: Built on top of generic `useApiCall`
- **Specific Interface**: Each hook has a focused purpose
- **Easy to Use**: Simple interface for components

#### 3. **useScrollToBottom Hook - UI Logic**

```typescript
// useScrollToBottom.ts
export function useScrollToBottom() {
  const scrollContainerRef = useRef<HTMLDivElement>(null);
  const [showScrollButton, setShowScrollButton] = useState(false);

  const scrollToBottom = useCallback(() => {
    if (scrollContainerRef.current) {
      scrollContainerRef.current.scrollTop = 
        scrollContainerRef.current.scrollHeight;
    }
  }, []);

  const checkScrollPosition = useCallback(() => {
    if (scrollContainerRef.current) {
      const { scrollTop, scrollHeight, clientHeight } = scrollContainerRef.current;
      const isAtBottom = scrollHeight - scrollTop - clientHeight < 50;
      setShowScrollButton(!isAtBottom);
    }
  }, []);

  useEffect(() => {
    const container = scrollContainerRef.current;
    if (container) {
      container.addEventListener('scroll', checkScrollPosition);
      return () => container.removeEventListener('scroll', checkScrollPosition);
    }
  }, [checkScrollPosition]);

  return { 
    scrollContainerRef, 
    showScrollButton, 
    scrollToBottom 
  };
}
```

**Why extract this logic:**
- **Reusable**: Can be used in any scrollable container
- **Complex Logic**: Scroll detection and management
- **Clean Components**: Components focus on rendering, not scroll logic
- **Testable**: Can test scroll behavior independently

### Using Custom Hooks in Components:

```typescript
// Chat.tsx
export function Chat() {
  const { state } = useChat();
  const { scrollContainerRef, showScrollButton, scrollToBottom } = useScrollToBottom();
  const { response, loading, error, sendMessage } = useSendMessage();

  const handleSendMessage = async (question: string) => {
    try {
      await sendMessage(question);
      scrollToBottom(); // Auto-scroll after sending
    } catch (err) {
      // Error handling is done in the hook
    }
  };

  return (
    <div className="chat-container">
      <div className="chat-messages" ref={scrollContainerRef}>
        {state.messages.map((message) => (
          <Message key={message.id} message={message} />
        ))}
        {loading && <LoadingMessage />}
      </div>
      
      {showScrollButton && (
        <button onClick={scrollToBottom} className="scroll-button">
          ↓
        </button>
      )}
      
      <ChatInput onSendMessage={handleSendMessage} />
    </div>
  );
}
```

### Custom Hook Best Practices:

#### 1. **Single Responsibility**
```typescript
// ❌ Bad - too many responsibilities
function useChatAndApi() {
  // Chat logic + API logic + scroll logic
}

// ✅ Good - focused responsibility
function useSendMessage() {
  // Only API logic for sending messages
}
```

#### 2. **Return Consistent Interface**
```typescript
// ✅ Consistent pattern across all API hooks
function useApiHook() {
  return {
    data: response,
    loading: boolean,
    error: string | null,
    execute: function
  };
}
```

#### 3. **Use useCallback for Functions**
```typescript
// ✅ Prevents unnecessary re-renders
const sendMessage = useCallback(async (question: string) => {
  return execute(() => apiService.sendMessage(question));
}, [execute]);
```

#### 4. **Handle Cleanup**
```typescript
// ✅ Clean up event listeners
useEffect(() => {
  const container = scrollContainerRef.current;
  if (container) {
    container.addEventListener('scroll', checkScrollPosition);
    return () => container.removeEventListener('scroll', checkScrollPosition);
  }
}, [checkScrollPosition]);
```

### Testing Custom Hooks:

```typescript
// useApi.test.ts
import { renderHook, act } from '@testing-library/react';
import { useSendMessage } from './useApi';

test('useSendMessage should handle loading state', async () => {
  const { result } = renderHook(() => useSendMessage());
  
  expect(result.current.loading).toBe(false);
  
  act(async () => {
    result.current.sendMessage('test question');
  });
  
  expect(result.current.loading).toBe(true);
});
```

### Benefits of Custom Hooks:

1. **Reusability**: Logic can be shared across components
2. **Testability**: Hooks can be tested in isolation
3. **Separation of Concerns**: UI logic separate from business logic
4. **Composability**: Hooks can be combined to create complex behavior
5. **Clean Components**: Components focus on rendering
6. **Type Safety**: Full TypeScript support
7. **Performance**: Proper memoization with useCallback/useMemo


## Q4: How do we implement TypeScript for type safety and better development experience?

**Answer:**

TypeScript integration is crucial for building maintainable, scalable React applications. Our Greek Derby chatbot uses TypeScript extensively to provide type safety, better IDE support, and self-documenting code.

### Why TypeScript in React?

**JavaScript Challenges:**
```javascript
// ❌ Runtime errors, no IDE support
function Message({ message, isError }) {
  return <div className={isError ? 'error' : 'normal'}>
    {message.text}
  </div>;
}

// What if message is undefined?
// What if isError is a string instead of boolean?
// No autocomplete for message properties
```

**TypeScript Benefits:**
```typescript
// ✅ Compile-time safety, full IDE support
interface MessageProps {
  message: Message;
  isError?: boolean;
}

function Message({ message, isError = false }: MessageProps) {
  return <div className={isError ? 'error' : 'normal'}>
    {message.text}
  </div>;
}
```

### Our TypeScript Configuration:

```json
// tsconfig.json
{
  "compilerOptions": {
    "target": "ES2022",
    "lib": ["DOM", "DOM.Iterable", "ES6"],
    "allowJs": true,
    "skipLibCheck": true,
    "esModuleInterop": true,
    "allowSyntheticDefaultImports": true,
    "strict": true,
    "forceConsistentCasingInFileNames": true,
    "module": "ESNext",
    "moduleResolution": "node",
    "resolveJsonModule": true,
    "isolatedModules": true,
    "noEmit": true,
    "jsx": "react-jsx"
  },
  "include": ["src"]
}
```

### Type Definitions:

#### 1. **Core Types**
```typescript
// types/index.ts
export interface Message {
  id: string;
  text: string;
  sender: 'user' | 'bot';
  timestamp: string;
  isError?: boolean;
}

export interface ChatState {
  messages: Message[];
  isLoading: boolean;
  sampleQuestions: string[];
  error: string | null;
}

export interface ChatResponse {
  answer: string;
  timestamp: string;
  conversation_id?: string;
}
```

#### 2. **API Types**
```typescript
// types/index.ts
export interface ApiError {
  message: string;
  status: number;
  statusText: string;
}

export interface SampleQuestionsResponse {
  sample_questions: string[];
  total_questions: number;
}

export interface ConversationHistory {
  history: Array<{
    user: string;
    bot: string;
  }>;
  total_messages: number;
}
```

#### 3. **Component Props Types**
```typescript
// types/index.ts
export interface MessageProps {
  message: Message;
  isError?: boolean;
}

export interface ChatInputProps {
  onSendMessage: (message: string) => void;
  disabled?: boolean;
  placeholder?: string;
}

export interface SampleQuestionsProps {
  questions: string[];
  onQuestionClick: (question: string) => void;
  loading?: boolean;
}
```

### Type-Safe API Service:

```typescript
// services/api.ts
export class ApiError extends Error {
  public status: number;
  public statusText: string;

  constructor(message: string, status: number, statusText: string) {
    super(message);
    this.name = 'ApiError';
    this.status = status;
    this.statusText = statusText;
  }
}

async function apiRequest<T>(
  endpoint: string,
  options: RequestInit = {}
): Promise<T> {
  const response = await fetch(`${API_BASE}${endpoint}`, {
    headers: {
      'Content-Type': 'application/json',
      ...options.headers,
    },
    ...options,
  });

  if (!response.ok) {
    throw new ApiError(
      `API Error: ${response.statusText}`,
      response.status,
      response.statusText
    );
  }

  return response.json();
}

export const apiService = {
  async sendMessage(question: string): Promise<ChatResponse> {
    return apiRequest<ChatResponse>('/chat', {
      method: 'POST',
      body: JSON.stringify({ question }),
    });
  },

  async getSampleQuestions(): Promise<SampleQuestionsResponse> {
    return apiRequest<SampleQuestionsResponse>('/sample-questions');
  }
};
```

### Type-Safe Context:

```typescript
// context/ChatContext.tsx
interface ChatContextType {
  state: ChatState;
  dispatch: React.Dispatch<ChatAction>;
  addMessage: (text: string, sender: 'user' | 'bot', isError?: boolean) => void;
  setLoading: (loading: boolean) => void;
  setError: (error: string | null) => void;
  clearMessages: () => void;
}

const ChatContext = createContext<ChatContextType | undefined>(undefined);

export function useChat(): ChatContextType {
  const context = useContext(ChatContext);
  if (context === undefined) {
    throw new Error('useChat must be used within a ChatProvider');
  }
  return context;
}
```

### Type-Safe Custom Hooks:

```typescript
// hooks/useApi.ts
function useApiCall<T>(): {
  data: T | null;
  loading: boolean;
  error: string | null;
  execute: (apiCall: () => Promise<T>) => Promise<T>;
  reset: () => void;
} {
  const [data, setData] = useState<T | null>(null);
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState<string | null>(null);

  const execute = useCallback(async (apiCall: () => Promise<T>): Promise<T> => {
    setLoading(true);
    setError(null);
    
    try {
      const result = await apiCall();
      setData(result);
      return result;
    } catch (err) {
      const errorMessage = err instanceof ApiError 
        ? err.message 
        : 'An unexpected error occurred';
      setError(errorMessage);
      throw err;
    } finally {
      setLoading(false);
    }
  }, []);

  const reset = useCallback(() => {
    setData(null);
    setError(null);
    setLoading(false);
  }, []);

  return { data, loading, error, execute, reset };
}
```

### Type-Safe Components:

```typescript
// components/Message.tsx
interface MessageProps {
  message: Message;
  isError?: boolean;
}

export function Message({ message, isError = false }: MessageProps) {
  return (
    <div className={`message ${message.sender} ${isError ? 'error' : ''}`}>
      <div className="message-content">
        {message.text}
      </div>
      <div className="message-timestamp">
        {message.timestamp}
      </div>
    </div>
  );
}
```

### TypeScript Best Practices:

#### 1. **Strict Type Checking**
```typescript
// tsconfig.json
{
  "compilerOptions": {
    "strict": true,
    "noImplicitAny": true,
    "strictNullChecks": true,
    "strictFunctionTypes": true
  }
}
```

#### 2. **Use Type Guards**
```typescript
function isApiError(error: unknown): error is ApiError {
  return error instanceof ApiError;
}

// Usage
try {
  await apiCall();
} catch (err) {
  if (isApiError(err)) {
    console.log(err.status, err.statusText);
  }
}
```

#### 3. **Generic Types for Reusability**
```typescript
interface ApiResponse<T> {
  data: T;
  status: number;
  message: string;
}

function useApi<T>(endpoint: string) {
  // Generic hook that works with any data type
}
```

#### 4. **Utility Types**
```typescript
// Make all properties optional
type PartialMessage = Partial<Message>;

// Pick specific properties
type MessagePreview = Pick<Message, 'text' | 'timestamp'>;

// Omit specific properties
type MessageWithoutId = Omit<Message, 'id'>;
```

### Benefits of TypeScript in Our Project:

1. **Compile-Time Safety**: Catch errors before runtime
2. **Better IDE Support**: Autocomplete, refactoring, navigation
3. **Self-Documenting Code**: Types serve as documentation
4. **Easier Refactoring**: Safe renaming and restructuring
5. **Better Team Collaboration**: Clear interfaces and contracts
6. **Reduced Bugs**: Type checking prevents common mistakes
7. **Enhanced Developer Experience**: Faster development with better tooling


## Q5: How do we implement performance optimization techniques?

**Answer:**

Performance optimization is crucial for creating smooth, responsive user experiences. Our Greek Derby chatbot implements several React performance optimization techniques to ensure fast rendering and efficient resource usage.

### Why Performance Matters in Chat Applications?

**Common Performance Issues:**
- **Slow Re-renders**: Every message causes full component tree re-render
- **Memory Leaks**: Event listeners and subscriptions not cleaned up
- **Expensive Calculations**: Complex operations on every render
- **Large Bundle Size**: Unnecessary code included in production
- **Network Inefficiency**: Multiple API calls for the same data

### Our Performance Optimization Strategies:

#### 1. **React.memo for Component Memoization**

```typescript
// components/Message.tsx
import { memo } from 'react';

interface MessageProps {
  message: Message;
  isError?: boolean;
}

export const Message = memo(function Message({ message, isError = false }: MessageProps) {
  return (
    <div className={`message ${message.sender} ${isError ? 'error' : ''}`}>
      <div className="message-content">
        {message.text}
      </div>
      <div className="message-timestamp">
        {message.timestamp}
      </div>
    </div>
  );
});

// Only re-render if message or isError changes
```

**Benefits:**
- **Prevents Unnecessary Re-renders**: Component only updates when props change
- **Better Performance**: Especially important for long message lists
- **Memory Efficiency**: Reduces React's reconciliation work

#### 2. **useCallback for Function Memoization**

```typescript
// hooks/useApi.ts
export function useSendMessage() {
  const { data, loading, error, execute } = useApiCall<ChatResponse>();

  // Memoize the function to prevent unnecessary re-renders
  const sendMessage = useCallback(async (question: string) => {
    return execute(() => apiService.sendMessage(question));
  }, [execute]);

  return { 
    response: data, 
    loading, 
    error, 
    sendMessage 
  };
}

// context/ChatContext.tsx
export function ChatProvider({ children }: ChatProviderProps) {
  const [state, dispatch] = useReducer(chatReducer, initialState);
  
  // Memoize action creators to prevent context value changes
  const addMessage = useCallback((text: string, sender: 'user' | 'bot', isError = false) => {
    const message: Message = {
      id: generateId(),
      text,
      sender,
      timestamp: new Date().toLocaleTimeString('el-GR'),
      isError
    };
    dispatch({ type: 'ADD_MESSAGE', payload: message });
  }, []);

  const value = useMemo(() => ({
    state,
    dispatch,
    addMessage,
    setLoading,
    setError,
    clearMessages,
  }), [state, addMessage, setLoading, setError, clearMessages]);

  return (
    <ChatContext.Provider value={value}>
      {children}
    </ChatContext.Provider>
  );
}
```

**Why useCallback:**
- **Stable References**: Functions don't change on every render
- **Prevents Child Re-renders**: Child components with memo won't re-render unnecessarily
- **Dependency Arrays**: Only recreate when dependencies change

#### 3. **useMemo for Expensive Calculations**

```typescript
// components/Chat.tsx
export function Chat() {
  const { state } = useChat();
  const { scrollContainerRef, showScrollButton, scrollToBottom } = useScrollToBottom();

  // Memoize filtered messages to avoid recalculation on every render
  const visibleMessages = useMemo(() => {
    return state.messages.filter(message => !message.isError);
  }, [state.messages]);

  // Memoize scroll position calculation
  const scrollPosition = useMemo(() => {
    if (scrollContainerRef.current) {
      const { scrollTop, scrollHeight, clientHeight } = scrollContainerRef.current;
      return {
        isAtBottom: scrollHeight - scrollTop - clientHeight < 50,
        scrollPercentage: (scrollTop / (scrollHeight - clientHeight)) * 100
      };
    }
    return { isAtBottom: true, scrollPercentage: 0 };
  }, [state.messages.length]); // Only recalculate when message count changes

  return (
    <div className="chat-container">
      <div className="chat-messages" ref={scrollContainerRef}>
        {visibleMessages.map((message) => (
          <Message key={message.id} message={message} />
        ))}
      </div>
      {/* ... */}
    </div>
  );
}
```

#### 4. **Virtual Scrolling for Large Lists**

```typescript
// For very large message lists (1000+ messages)
import { FixedSizeList as List } from 'react-window';

function VirtualizedMessageList({ messages }: { messages: Message[] }) {
  const Row = ({ index, style }: { index: number; style: React.CSSProperties }) => (
    <div style={style}>
      <Message message={messages[index]} />
    </div>
  );

  return (
    <List
      height={600}
      itemCount={messages.length}
      itemSize={80}
      width="100%"
    >
      {Row}
    </List>
  );
}
```

#### 5. **Lazy Loading and Code Splitting**

```typescript
// App.tsx - Lazy load heavy components
import { lazy, Suspense } from 'react';

const Chat = lazy(() => import('./components/Chat'));
const SampleQuestions = lazy(() => import('./components/SampleQuestions'));

function App() {
  return (
    <ChatProvider>
      <div className="container">
        <Header />
        <Suspense fallback={<div>Loading chat...</div>}>
          <Chat />
        </Suspense>
        <Suspense fallback={<div>Loading questions...</div>}>
          <SampleQuestions />
        </Suspense>
      </div>
    </ChatProvider>
  );
}
```

#### 6. **Debouncing for User Input**

```typescript
// hooks/useDebounce.ts
import { useState, useEffect } from 'react';

export function useDebounce<T>(value: T, delay: number): T {
  const [debouncedValue, setDebouncedValue] = useState<T>(value);

  useEffect(() => {
    const handler = setTimeout(() => {
      setDebouncedValue(value);
    }, delay);

    return () => {
      clearTimeout(handler);
    };
  }, [value, delay]);

  return debouncedValue;
}

// components/ChatInput.tsx
export function ChatInput({ onSendMessage }: ChatInputProps) {
  const [input, setInput] = useState('');
  const debouncedInput = useDebounce(input, 300);

  // Only trigger search/validation after user stops typing
  useEffect(() => {
    if (debouncedInput.length > 2) {
      // Validate input or show suggestions
    }
  }, [debouncedInput]);
}
```

#### 7. **Bundle Optimization**

```typescript
// vite.config.ts
export default defineConfig({
  plugins: [react()],
  build: {
    outDir: 'dist',
    sourcemap: false,
    rollupOptions: {
      output: {
        manualChunks: {
          vendor: ['react', 'react-dom'],
          api: ['axios', 'fetch'],
        },
      },
    },
  },
  server: {
    host: '0.0.0.0',
    port: 5173,
    proxy: {
      '/api': {
        target: 'http://backend:8000',
        changeOrigin: true,
        rewrite: (path) => path.replace(/^\/api/, '')
      }
    }
  }
});
```

### Performance Monitoring:

```typescript
// utils/performance.ts
export function measurePerformance(name: string, fn: () => void) {
  const start = performance.now();
  fn();
  const end = performance.now();
  console.log(`${name} took ${end - start} milliseconds`);
}

// Usage in components
const handleSendMessage = useCallback(async (question: string) => {
  measurePerformance('sendMessage', async () => {
    await sendMessage(question);
  });
}, [sendMessage]);
```

### React DevTools Profiler:

```typescript
// Wrap components for profiling
import { Profiler } from 'react';

function onRenderCallback(id, phase, actualDuration) {
  console.log('Render:', { id, phase, actualDuration });
}

<Profiler id="Chat" onRender={onRenderCallback}>
  <Chat />
</Profiler>
```

### Best Practices Summary:

1. **Use React.memo** for components that receive stable props
2. **Use useCallback** for functions passed as props
3. **Use useMemo** for expensive calculations
4. **Implement virtual scrolling** for large lists
5. **Use code splitting** for route-based lazy loading
6. **Debounce user input** to reduce API calls
7. **Optimize bundle size** with manual chunking
8. **Monitor performance** with React DevTools
9. **Clean up effects** to prevent memory leaks
10. **Use proper dependency arrays** in hooks

### Performance Metrics to Track:

- **First Contentful Paint (FCP)**: When first content appears
- **Largest Contentful Paint (LCP)**: When main content loads
- **Time to Interactive (TTI)**: When page becomes interactive
- **Bundle Size**: JavaScript bundle size
- **Re-render Count**: How often components re-render
- **Memory Usage**: Memory consumption over time


## Q6: How do we implement modern CSS and responsive design?

**Answer:**

Modern CSS and responsive design are essential for creating accessible, user-friendly interfaces that work across all devices. Our Greek Derby chatbot uses modern CSS techniques, CSS modules, and responsive design principles to ensure an optimal user experience.

### Modern CSS Architecture:

#### 1. **CSS Modules for Scoped Styles**

```css
/* Message.module.css */
.message {
  display: flex;
  flex-direction: column;
  margin-bottom: 1rem;
  padding: 0.75rem;
  border-radius: 0.5rem;
  max-width: 80%;
  word-wrap: break-word;
  animation: slideIn 0.3s ease-out;
}

.message.user {
  align-self: flex-end;
  background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
  color: white;
  margin-left: auto;
}

.message.bot {
  align-self: flex-start;
  background: #f8f9fa;
  color: #333;
  border: 1px solid #e9ecef;
}

.message.error {
  background: #f8d7da;
  color: #721c24;
  border: 1px solid #f5c6cb;
}

.messageContent {
  font-size: 0.95rem;
  line-height: 1.4;
  margin-bottom: 0.25rem;
}

.messageTimestamp {
  font-size: 0.75rem;
  opacity: 0.7;
  text-align: right;
}

@keyframes slideIn {
  from {
    opacity: 0;
    transform: translateY(10px);
  }
  to {
    opacity: 1;
    transform: translateY(0);
  }
}

/* Responsive design */
@media (max-width: 768px) {
  .message {
    max-width: 90%;
    padding: 0.5rem;
  }
  
  .messageContent {
    font-size: 0.9rem;
  }
}
```

#### 2. **CSS Custom Properties (Variables)**

```css
/* App.css - Global CSS variables */
:root {
  /* Colors */
  --primary-color: #667eea;
  --secondary-color: #764ba2;
  --success-color: #28a745;
  --error-color: #dc3545;
  --warning-color: #ffc107;
  --info-color: #17a2b8;
  
  /* Grays */
  --gray-100: #f8f9fa;
  --gray-200: #e9ecef;
  --gray-300: #dee2e6;
  --gray-400: #ced4da;
  --gray-500: #adb5bd;
  --gray-600: #6c757d;
  --gray-700: #495057;
  --gray-800: #343a40;
  --gray-900: #212529;
  
  /* Spacing */
  --spacing-xs: 0.25rem;
  --spacing-sm: 0.5rem;
  --spacing-md: 1rem;
  --spacing-lg: 1.5rem;
  --spacing-xl: 2rem;
  --spacing-xxl: 3rem;
  
  /* Typography */
  --font-family: 'Inter', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
  --font-size-xs: 0.75rem;
  --font-size-sm: 0.875rem;
  --font-size-base: 1rem;
  --font-size-lg: 1.125rem;
  --font-size-xl: 1.25rem;
  --font-size-2xl: 1.5rem;
  
  /* Shadows */
  --shadow-sm: 0 1px 2px 0 rgba(0, 0, 0, 0.05);
  --shadow-md: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
  --shadow-lg: 0 10px 15px -3px rgba(0, 0, 0, 0.1);
  --shadow-xl: 0 20px 25px -5px rgba(0, 0, 0, 0.1);
  
  /* Border radius */
  --radius-sm: 0.25rem;
  --radius-md: 0.375rem;
  --radius-lg: 0.5rem;
  --radius-xl: 0.75rem;
  --radius-full: 9999px;
  
  /* Transitions */
  --transition-fast: 150ms ease-in-out;
  --transition-normal: 300ms ease-in-out;
  --transition-slow: 500ms ease-in-out;
}

/* Dark mode support */
@media (prefers-color-scheme: dark) {
  :root {
    --gray-100: #1a1a1a;
    --gray-200: #2d2d2d;
    --gray-300: #404040;
    --gray-400: #525252;
    --gray-500: #737373;
    --gray-600: #a3a3a3;
    --gray-700: #d4d4d4;
    --gray-800: #e5e5e5;
    --gray-900: #f5f5f5;
  }
}
```

### Responsive Design Strategy:

#### 1. **Mobile-First Approach**

```css
/* Base styles for mobile */
.container {
  width: 100%;
  padding: var(--spacing-sm);
}

.chatMessages {
  padding: var(--spacing-sm);
}

.message {
  max-width: 90%;
  font-size: var(--font-size-sm);
}

/* Tablet styles */
@media (min-width: 768px) {
  .container {
    max-width: 768px;
    margin: 0 auto;
    padding: var(--spacing-md);
  }
  
  .chatMessages {
    padding: var(--spacing-md);
  }
  
  .message {
    max-width: 80%;
    font-size: var(--font-size-base);
  }
}

/* Desktop styles */
@media (min-width: 1024px) {
  .container {
    max-width: 1024px;
    padding: var(--spacing-lg);
  }
  
  .chatMessages {
    padding: var(--spacing-lg);
  }
  
  .message {
    max-width: 70%;
  }
}
```

#### 2. **Touch-Friendly Design**

```css
/* Touch targets */
.touchTarget {
  min-height: 44px;
  min-width: 44px;
  display: flex;
  align-items: center;
  justify-content: center;
}

.button {
  padding: var(--spacing-sm) var(--spacing-md);
  border-radius: var(--radius-md);
  font-size: var(--font-size-base);
  cursor: pointer;
  transition: all var(--transition-fast);
  user-select: none;
  -webkit-tap-highlight-color: transparent;
}

.button:active {
  transform: scale(0.98);
}

/* Sample questions as touch targets */
.sampleQuestion {
  display: block;
  padding: var(--spacing-sm) var(--spacing-md);
  margin-bottom: var(--spacing-sm);
  background: white;
  border: 1px solid var(--gray-200);
  border-radius: var(--radius-lg);
  text-align: left;
  cursor: pointer;
  transition: all var(--transition-fast);
  min-height: 44px;
}

.sampleQuestion:hover {
  background: var(--gray-100);
  border-color: var(--primary-color);
  transform: translateY(-1px);
  box-shadow: var(--shadow-sm);
}
```

### Modern CSS Features:

#### 1. **CSS Grid for Complex Layouts**

```css
/* Header layout with CSS Grid */
.header {
  display: grid;
  grid-template-columns: auto 1fr auto;
  grid-template-areas: "logo title actions";
  align-items: center;
  gap: var(--spacing-md);
  padding: var(--spacing-md);
  background: white;
  box-shadow: var(--shadow-sm);
}

.logo {
  grid-area: logo;
}

.title {
  grid-area: title;
  text-align: center;
}

.actions {
  grid-area: actions;
  display: flex;
  gap: var(--spacing-sm);
}

/* Responsive grid */
@media (max-width: 768px) {
  .header {
    grid-template-columns: 1fr auto;
    grid-template-areas: 
      "title actions"
      "logo .";
    text-align: left;
  }
}
```

#### 2. **CSS Animations and Transitions**

```css
/* Loading animation */
.loadingDots {
  display: inline-flex;
  gap: 4px;
}

.loadingDot {
  width: 8px;
  height: 8px;
  border-radius: 50%;
  background: var(--primary-color);
  animation: loadingBounce 1.4s ease-in-out infinite both;
}

.loadingDot:nth-child(1) { animation-delay: -0.32s; }
.loadingDot:nth-child(2) { animation-delay: -0.16s; }
.loadingDot:nth-child(3) { animation-delay: 0s; }

@keyframes loadingBounce {
  0%, 80%, 100% {
    transform: scale(0);
  }
  40% {
    transform: scale(1);
  }
}

/* Smooth transitions */
.fadeIn {
  animation: fadeIn 0.3s ease-in-out;
}

@keyframes fadeIn {
  from {
    opacity: 0;
    transform: translateY(10px);
  }
  to {
    opacity: 1;
    transform: translateY(0);
  }
}
```

### Accessibility Considerations:

```css
/* Focus indicators */
.focusable:focus {
  outline: 2px solid var(--primary-color);
  outline-offset: 2px;
}

/* Reduced motion support */
@media (prefers-reduced-motion: reduce) {
  * {
    animation-duration: 0.01ms !important;
    animation-iteration-count: 1 !important;
    transition-duration: 0.01ms !important;
  }
}

/* High contrast mode */
@media (prefers-contrast: high) {
  .message {
    border: 2px solid currentColor;
  }
  
  .button {
    border: 2px solid currentColor;
  }
}
```

### Best Practices Summary:

1. **Use CSS Modules** for scoped styles
2. **Implement CSS Custom Properties** for theming
3. **Mobile-first responsive design**
4. **Touch-friendly interface** (44px minimum touch targets)
5. **Flexible typography** with clamp()
6. **Hardware-accelerated animations**
7. **Accessibility considerations** (focus, reduced motion, high contrast)
8. **Performance optimizations** (efficient selectors, will-change)
9. **Consistent spacing** with CSS variables
10. **Modern CSS features** (Grid, Flexbox, Custom Properties)

---

## 🎯 Summary

This lesson covered the essential React frontend concepts used in our Greek Derby RAG chatbot:

### Key Takeaways:

1. **Modern React Patterns**: Component composition, custom hooks, and Context API
2. **State Management**: useReducer + Context for complex state without Redux
3. **Custom Hooks**: Reusable logic for API calls, scrolling, and UI interactions
4. **TypeScript Integration**: Type safety, better IDE support, and maintainable code
5. **Performance Optimization**: Memoization, lazy loading, and efficient rendering
6. **Modern CSS**: Responsive design, CSS variables, and accessibility

### Next Steps:

- **Practice**: Try building your own React components using these patterns
- **Explore**: Learn about React Query for server state management
- **Advanced**: Study React 18 features like Concurrent Rendering and Suspense
- **Testing**: Implement unit tests for your components and hooks

### Project Structure:

```
front-end/react-chatbot/
├── src/
│   ├── components/          # Reusable UI components
│   ├── context/            # Global state management
│   ├── hooks/              # Custom hooks for logic reuse
│   ├── services/           # API integration layer
│   ├── types/              # TypeScript type definitions
│   └── App.tsx             # Main application component
├── public/                 # Static assets
├── package.json            # Dependencies and scripts
├── tsconfig.json           # TypeScript configuration
└── vite.config.ts          # Build tool configuration
```

This architecture provides a solid foundation for building scalable, maintainable React applications! 🚀
